## Introduction

For our project We are analyzing the dataset of all flights that departed from the three main New York City airports in 2023. For this project We will be analyzing all the features of the dataset and from the auxiliary ones. Our target variable for regression is 'arr_delay' and for classification we have chosen cancellation prediction. When we don't have information about departure or arrival time it means that the flight was cancelled.

## Data Cleaning

Overview of datasets:
* airlines.csv
    * `carrier` -- Two-letter airline code.
    * `name` -- Full name of the airline.

* airports.csv
    * `faa` -- FAA code of the airport (e.g., JFK for John F. Kennedy).
    * `name` -- Name of the airport.
    * `lat` -- Latitude of the airport (decimal format).
    * `lon` -- Longitude of the airport (decimal format).
    * `alt` -- Altitude of the airport in feet.
    * `tz` -- Time zone offset (UTC offset). Can be `null`
    * `dst` -- Daylight saving time indicator (`A` = Active, etc.). Can be `null` but it is `null` iff `tz` is `null`
    * `tzone` -- IANA time zone name (e.g., America/New_York). Can be `null` but it is `null` in superset of cases when `tz` is null, therefore can be filled sometimes with already existing values

* flights.csv
    * `year` -- Year of the flight.
    * `month` -- Month of the flight.
    * `day` -- Day of the flight.
    * `dep_time` -- Actual departure time (local, in military format). Can be `null`
    * `sched_dep_time` -- Scheduled departure time (local, in military format).
    * `dep_delay` -- Departure delay in minutes (negative if early). Can be `null` iff `dep_time` is null
    * `arr_time` -- Actual arrival time (local, in military format). Can be `null` in superset of cases when `dep_delay` is `null`. If it is `null` we assume that flight was cancelled.
    * `sched_arr_time` -- Scheduled arrival time (local, in military format).
    * `arr_delay` -- Arrival delay in minutes (negative if early). Can be `null` in superset of cases when `arr_time` is `null`. Can be filled in as difference between `sched_arr_time` and `arr_time`
    * `carrier` -- Two-letter airline code (links to `airlines.csv`).
    * `flight` -- Flight number.
    * `tailnum` -- Aircraft tail number (links to `planes.csv`). Cna be `null` but number is small => can be dropped.
    * `origin` -- Origin airport code (links to `airports.csv`).
    * `dest` -- Destination airport code (links to `airports.csv`).
    * `air_time` -- Total air time in minutes. Can be `null` iff `arr_delay` is null but we decided not to use this column => can be ignored.
    * `distance` -- Distance of the flight in miles.
    * `hour` -- Scheduled departure hour (derived from `sched_dep_time`).
    * `minute` -- Scheduled departure minute (derived from `sched_dep_time`).
    * `time_hour` -- Rounded time to the nearest hour (useful for joining with weather data).

* planes.csv
    * `tailnum` -- Unique aircraft identifier (matches with `flights.tailnum`).
    * `year` -- Year the plane was manufactured. Can be `null` but it is quite rare case, therefore can be dropped
    * `type` -- Aircraft type (e.g., "Fixed wing multi engine").
    * `manufacturer` -- Manufacturer of the aircraft (e.g., Boeing, Airbus).
    * `model` -- Model of the aircraft (e.g., "A320").
    * `engines` -- Number of engines on the aircraft.
    * `seats` -- Number of passenger seats on the plane.
    * `speed` -- Typical cruise speed (if available; may be NaN).
    * `engine` -- Type of engine (e.g., "Turbo-fan").

* weather.csv
    * `origin` -- Airport code where the weather data is recorded (links to `airports.faa`).
    * `year` -- Year of the weather record.
    * `month` -- Month of the weather record.
    * `day` -- Day of the weather record.
    * `hour` -- Hour of the weather observation (local).
    * `temp` -- Temperature in degrees Fahrenheit. Can be `null` in superset of cases when `precip` is `null`
    * `dewp` -- Dewpoint temperature in degrees Fahrenheit. Can be `null` iff `temp` is `null`
    * `humid` -- Relative humidity (%). Can be `null` iff `temp` is `null`
    * `wind_dir` -- Wind direction in degrees (0° = North, 90° = East, etc.).
    * `wind_speed` -- Wind speed in mph.
    * `wind_gust` -- Wind gust speed in mph (if available).
    * `precip` -- Precipitation in inches. Can be `null`.
    * `pressure` -- Atmospheric pressure in millibars (if available). Can be `null` in superset of cases when `temp` is `null`
    * `visib` -- Visibility in miles.
    * `time_hour` -- Time of the observation rounded to the nearest hour (links to `flights.time_hour`).



### Imports

In [ ]:
!pip install -r requirements.txt

In [ ]:
import pandas as pd

### Load datasets

In [ ]:
def read_csv(filename):
    df = pd.read_csv(
        filename,
        header=0,                     # Use the first row as the header
        delimiter=';',                # Use semicolon as the main delimiter
        decimal=',',                  # Specify that commas are used as decimals
        quotechar='"',                # Handle quotes around strings
        skipinitialspace=True,        # Skip spaces after delimiters
    )
    df.set_index(df.columns[0], inplace=True)
    return df

airlines = read_csv('airlines.csv')
airports = read_csv('airports.csv')
planes = read_csv('planes.csv')
flights = read_csv('flights.csv')
weather = read_csv('weather.csv')

In [ ]:
planes['year'] = planes['year'].astype('Int64')
flights['dep_time'] = flights['dep_time'].astype('Int64')
flights['dep_delay'] = flights['dep_delay'].astype('Int64')
flights['arr_time'] = flights['arr_time'].astype('Int64')
flights['arr_delay'] = flights['arr_delay'].astype('Int64')

In [ ]:
def check_for_nulls(df):
    res = {}
    for c in df.columns:
        if df[df[c].isnull()].shape[0] != 0:
            res[c] = df[df[c].isnull()].shape[0]
    return res

In [ ]:
print("Nullable columns in airlines:", check_for_nulls(airlines))
print("Nullable columns in airports:", check_for_nulls(airports))
print("Nullable columns in planes:", check_for_nulls(planes))
print("Nullable columns in flights:", check_for_nulls(flights))
print("Nullable columns in weather:", check_for_nulls(weather))

### Clean Datasets

In [ ]:
def clear_airports(df): # fill `tzone` if can be filled, drop otherwise. If `tz` is unknown, drop
    df = df.dropna(subset=['tz', 'dst'])

    for idx, row in df[df['tzone'].isnull()].iterrows():
        matching_row = df[(df['tz'] == row['tz']) & pd.notnull(df['tzone'])]
        if not matching_row.empty:
            df.at[idx, 'tzone'] = matching_row['tzone'].iloc[0]
        else:
            df = df.drop(idx)
    return df

In [ ]:
def clear_planes(df): # drop all planes if `year` is unknown
    return df.dropna()

In [ ]:
def clear_flights(df): # drop if `tailnum` is unknown, fix military format of time, fill air_time where needed
    df = df.dropna(subset=['tailnum'])

    for idx, row in df.iterrows():
        df.at[idx, 'dep_time'] = df.at[idx, 'dep_time'] % 100 + (df.at[idx, 'dep_time'] // 100) * 60
        df.at[idx, 'arr_time'] = df.at[idx, 'arr_time'] % 100 + (df.at[idx, 'arr_time'] // 100) * 60
        df.at[idx, 'sched_dep_time'] = df.at[idx, 'sched_dep_time'] % 100 + (df.at[idx, 'sched_dep_time'] // 100) * 60
        df.at[idx, 'sched_arr_time'] = df.at[idx, 'sched_arr_time'] % 100 + (df.at[idx, 'sched_arr_time'] // 100) * 60

    for idx, row in df[df['air_time'].isnull()].iterrows():
        df.at[idx, 'arr_delay'] = df['arr_time'].iloc[0] - df['sched_arr_time'].iloc[0]
        df.at[idx, 'air_time'] = df['arr_time'].iloc[0] - df['dep_time'].iloc[0]
    return df

In [ ]:
def clear_weather(df): # here we ignore all nulls and keep them as is
    return df

In [ ]:
clean_airlines = airlines
clean_airports = clear_airports(airports.copy())
clean_planes = clear_planes(planes.copy())
clean_flights = clear_flights(flights.copy())
clean_weather = clear_weather(weather.copy())

In [ ]:
print("Nullable columns in airlines:", check_for_nulls(clean_airlines))
print("Nullable columns in airports:", check_for_nulls(clean_airports))
print("Nullable columns in planes:", check_for_nulls(clean_planes))
print("Nullable columns in flights:", check_for_nulls(clean_flights))
print("Nullable columns in weather:", check_for_nulls(clean_weather))

In [ ]:
print("Airports old vs new:", airports.shape, clean_airports.shape)
print("Planes old vs new:", planes.shape, clean_planes.shape)
print("Flights old vs new:", flights.shape, clean_flights.shape)
print("Weather old vs new:", weather.shape, clean_weather.shape)

### Merge datasets

In [ ]:
# merge flights with info about origin airport
flights_with_origin_info = pd.merge(
    clean_flights,
    clean_airports,
    how='left',
    left_on='origin',
    right_on='faa',
    suffixes=('', '_origin')
)

# merge obtained dataset with info about destination airport
flights_with_origin_and_dest = pd.merge(
    flights_with_origin_info,
    clean_airports,
    how='inner',
    left_on='dest',
    right_on='faa',
    suffixes=('_origin', '_dest')
)

# merge obtained dataset with info about plane
flights_with_origin_dest_and_planes = pd.merge(
    flights_with_origin_and_dest,
    clean_planes,
    how='left',
    left_on=['tailnum'],
    right_on=['tailnum'],
    suffixes=('', '_plane')
)

# merge obtained dataset with info about weather at the departure time
merged_df = pd.merge(
    flights_with_origin_dest_and_planes,
    clean_weather,
    how='left',
    left_on=['origin', 'year', 'month', 'day', 'hour'],
    right_on=['origin', 'year', 'month', 'day', 'hour'],
    suffixes=('', '_weather')
)

In [ ]:
# drop columns that are not needed for classification and regression
merged_df = merged_df.drop(['dep_time', 'arr_time', 'flight', 'tailnum', 'air_time', 'time_hour', 'faa_origin', 'name_origin', 'lat_origin', 'lon_origin', 'alt_origin', 'tz_origin', 'dst_origin', 'tzone_origin', 'faa_dest', 'name_dest', 'dst_dest', 'tzone_dest', 'time_hour_weather'], axis=1)


In [ ]:
merged_df.to_csv("merged_df.csv", index=False)

In [ ]:
# split into 3 datasets depending on origin airport
ewr_df = merged_df[merged_df['origin'] == 'EWR'].drop('origin', axis=1)
jfk_df = merged_df[merged_df['origin'] == 'JFK'].drop('origin', axis=1)
lga_df = merged_df[merged_df['origin'] == 'LGA'].drop('origin', axis=1)

In [ ]:
# process each dataset and create datasets for both classification and regression task
ewr_delay_df = ewr_df.dropna(subset=['arr_delay'])
ewr_cancel_df = ewr_df
ewr_cancel_df['is_cancelled'] = ewr_cancel_df['dep_delay'].isnull()
ewr_cancel_df = ewr_cancel_df.drop(['dep_delay', 'arr_delay'], axis=1)
jfk_delay_df = jfk_df.dropna(subset=['arr_delay'])
jfk_cancel_df = jfk_df
jfk_cancel_df['is_cancelled'] = jfk_cancel_df['dep_delay'].isnull()
jfk_cancel_df = jfk_cancel_df.drop(['dep_delay', 'arr_delay'], axis=1)
lga_delay_df = lga_df.dropna(subset=['arr_delay'])
lga_cancel_df = lga_df
lga_cancel_df['is_cancelled'] = lga_cancel_df['dep_delay'].isnull()
lga_cancel_df = lga_cancel_df.drop(['dep_delay', 'arr_delay'], axis=1)

In [ ]:
ewr_delay_df.to_csv("ewr_delay.csv", index=False)
jfk_delay_df.to_csv("jfk_delay.csv", index=False)
lga_delay_df.to_csv("lga_delay.csv", index=False)
ewr_cancel_df.to_csv("ewr_cancel.csv", index=False)
jfk_cancel_df.to_csv("jfk_cancel.csv", index=False)
lga_cancel_df.to_csv("lga_cancel.csv", index=False)